# Multi-Objective Reservoir Optimization Showcase

This notebook demonstrates multi-objective optimization for a realistic water system using taqsim.

**System Configuration:**
- **72 optimizable parameters**: 60 SLOP release rule + 12 SeasonalRatio split rule
- **4 objectives**: maximize hydropower, minimize city flooding, minimize irrigation deficit, minimize thermal deficit
- **120 timesteps**: 10 years × 12 months
- **9 nodes, 8 edges**: realistic water network topology

The optimization uses NSGA-II to find Pareto-optimal trade-offs between competing objectives.

In [ ]:
from dataclasses import dataclass
from typing import ClassVar

import matplotlib.pyplot as plt
import numpy as np

from taqsim import (
    Demand,
    Edge,
    LossReason,
    Objective,
    Ordered,
    PassThrough,
    Sink,
    Source,
    Splitter,
    Storage,
    Strategy,
    TimeSeries,
    WaterSystem,
    lift,
    optimize,
)
from taqsim.node.events import WaterPassedThrough, WaterSpilled, WaterStored

TIMESTEPS = 120
SEED = 96530  # From exploration trial 0

## Time Series Generation

Generate realistic seasonal patterns for river inflows and demands:
- **Inflows**: Peak in summer (months 5-7), low in winter, with random variability and occasional extreme events
- **Irrigation demand**: Peak in summer growing season (months 6-8), zero in winter
- **Thermal demand**: Higher in summer (more cooling needed), relatively stable baseline

In [ ]:
# Configuration from exploration trial 0
INFLOW_SCALE = 0.89
IRRIGATION_PEAK = 25.4
THERMAL_BASE = 24.1
CITY_CAPACITY = 41.0


def generate_inflows(n_years: int = 10, mean_annual: float = 100.0, seed: int = 42) -> list[float]:
    """Generate monthly river inflows with seasonal pattern and variability."""
    rng = np.random.default_rng(seed)
    seasonal = [0.5, 0.6, 0.8, 1.2, 1.6, 1.8, 1.5, 1.0, 0.7, 0.5, 0.4, 0.4]

    inflows = []
    for year in range(n_years):
        year_factor = rng.uniform(0.8, 1.2)
        for month, s in enumerate(seasonal):
            noise = rng.uniform(0.9, 1.1)
            extreme = 1.8 if rng.random() < 0.08 else 1.0
            inflows.append(mean_annual * s * year_factor * noise * extreme)
    return inflows


def generate_irrigation_demand(n_years: int = 10, peak_demand: float = 25.0, seed: int = 123) -> list[float]:
    """Generate monthly irrigation demand with growing season pattern."""
    rng = np.random.default_rng(seed)
    seasonal = [0.0, 0.0, 0.2, 0.6, 1.0, 1.4, 1.5, 1.2, 0.7, 0.2, 0.0, 0.0]

    demand = []
    for year in range(n_years):
        for month, s in enumerate(seasonal):
            noise = rng.uniform(0.9, 1.1)
            demand.append(peak_demand * s * noise)
    return demand


def generate_thermal_demand(n_years: int = 10, base_demand: float = 24.0, seed: int = 456) -> list[float]:
    """Generate thermal plant cooling demand - higher in summer when more cooling is needed."""
    rng = np.random.default_rng(seed)
    seasonal = [0.7, 0.7, 0.8, 0.9, 1.1, 1.3, 1.4, 1.3, 1.1, 0.9, 0.8, 0.7]

    demand = []
    for year in range(n_years):
        for month, s in enumerate(seasonal):
            noise = rng.uniform(0.95, 1.05)
            demand.append(base_demand * s * noise)
    return demand


inflows = generate_inflows(n_years=10, mean_annual=100.0 * INFLOW_SCALE, seed=SEED)
irrigation_demand = generate_irrigation_demand(n_years=10, peak_demand=IRRIGATION_PEAK, seed=SEED + 1)
thermal_demand = generate_thermal_demand(n_years=10, base_demand=THERMAL_BASE, seed=SEED + 2)

print(f"Generated {len(inflows)} timesteps of data")
print(f"Total inflow: {sum(inflows):.1f}, Mean: {np.mean(inflows):.1f}")
print(f"Total irrigation demand: {sum(irrigation_demand):.1f}, Mean: {np.mean(irrigation_demand):.1f}")
print(f"Total thermal demand: {sum(thermal_demand):.1f}, Mean: {np.mean(thermal_demand):.1f}")

## Custom Strategies

### SLOP Release Rule (60 parameters)
Storage Level Operating Policy with time-varying parameters. For each of 12 months, defines:
- `h1`, `h2`: Water level thresholds (low and high)
- `w`: Base release rate
- `m1`: Slope below h1 (conservation zone)
- `m2`: Slope above h2 (flood control zone)

### SeasonalRatio Split Rule (12 parameters)
Monthly varying split between irrigation and thermal plant cooling:
- `irrigation_fraction`: Fraction allocated to irrigation (remainder to thermal)

In [ ]:
def volume_to_head(
    volume: float, v_dead: float = 10.0, v_max: float = 150.0, h_dead: float = 20.0, h_max: float = 100.0
) -> float:
    """Convert reservoir volume to hydraulic head (water level)."""
    v_clamped = max(v_dead, min(v_max, volume))
    return h_dead + (v_clamped - v_dead) * (h_max - h_dead) / (v_max - v_dead)


@lift
def volume_to_head_lifted(v: float) -> float:
    """Lifted version for trace arithmetic."""
    return 20.0 + (max(10.0, min(150.0, v)) - 10.0) * 80.0 / 140.0


@dataclass(frozen=True)
class SLOPRelease(Strategy):
    """Storage Level Operating Policy with time-varying monthly parameters."""

    __params__: ClassVar[tuple[str, ...]] = ("h1", "h2", "w", "m1", "m2")
    __bounds__: ClassVar[dict[str, tuple[float, float]]] = {
        "h1": (20.0, 100.0),
        "h2": (20.0, 100.0),
        "w": (0.0, 80.0),
        "m1": (0.01, 2.0),
        "m2": (0.01, 2.0),
    }
    __constraints__: ClassVar[tuple] = (Ordered(low="h1", high="h2"),)
    __time_varying__: ClassVar[tuple[str, ...]] = ("h1", "h2", "w", "m1", "m2")
    __cyclical__: ClassVar[tuple[str, ...]] = ("h1", "h2", "w", "m1", "m2")

    h1: tuple[float, ...] = (40.0,) * 12
    h2: tuple[float, ...] = (70.0,) * 12
    w: tuple[float, ...] = (40.0,) * 12
    m1: tuple[float, ...] = (0.5,) * 12
    m2: tuple[float, ...] = (0.8,) * 12

    def release(self, node: Storage, inflow: float, t: int, dt: float) -> float:
        head = volume_to_head(node.storage)
        month = t % 12

        h1_t = self.h1[month]
        h2_t = self.h2[month]
        w_t = self.w[month]
        m1_t = self.m1[month]
        m2_t = self.m2[month]

        # Piecewise linear SLOP policy
        if head < h1_t:
            # Conservation zone: reduce release linearly
            release = max(0.0, w_t - m1_t * (h1_t - head))
        elif head > h2_t:
            # Flood control zone: increase release linearly
            release = w_t + m2_t * (head - h2_t)
        else:
            # Normal zone: release at base rate
            release = w_t

        # Cannot release more than available (above dead storage)
        available = max(0.0, node.storage - node.dead_storage)
        return min(release * dt, available)


@dataclass(frozen=True)
class SeasonalRatio(Strategy):
    """Time-varying split ratio between irrigation and thermal plant."""

    __params__: ClassVar[tuple[str, ...]] = ("irrigation_fraction",)
    __bounds__: ClassVar[dict[str, tuple[float, float]]] = {"irrigation_fraction": (0.1, 0.9)}
    __time_varying__: ClassVar[tuple[str, ...]] = ("irrigation_fraction",)
    __cyclical__: ClassVar[tuple[str, ...]] = ("irrigation_fraction",)

    irrigation_fraction: tuple[float, ...] = (0.5,) * 12

    def split(self, node: Splitter, amount: float, t: int) -> dict[str, float]:
        frac = self.irrigation_fraction[t % 12]
        return {
            "irrigation": amount * frac,
            "thermal_plant": amount * (1.0 - frac),
        }


# Count optimizable parameters
slop_params = 5 * 12  # 5 params × 12 months
ratio_params = 1 * 12  # 1 param × 12 months
print(f"SLOP parameters: {slop_params}")
print(f"SeasonalRatio parameters: {ratio_params}")
print(f"Total optimizable parameters: {slop_params + ratio_params}")

## Custom Objectives

Define four objectives for multi-objective optimization:

1. **Hydropower**: Maximize power generation (P = ρ × g × Q × H × η)
2. **City Flood Risk**: Minimize spillage at city passthrough (capacity overflow)
3. **Irrigation Deficit**: Minimize unmet irrigation demand
4. **Thermal Deficit**: Minimize unmet thermal plant cooling demand

**The trade-offs:**
- **Hydropower vs City Flood**: High releases generate more power but risk flooding the city
- **Irrigation vs Thermal**: Water allocation is zero-sum - more to one means less to the other
- **Release vs Deficits**: Must release enough to meet downstream demands

In [ ]:
from taqsim.node.events import WaterReleased, DeficitRecorded


def hydropower_objective(
    reservoir_id: str, turbine_id: str, initial_storage: float = 75.0, efficiency: float = 0.85
) -> Objective:
    """Create objective to maximize hydropower generation using dynamic head."""

    def evaluate(system: WaterSystem) -> float:
        turbine = system.nodes[turbine_id]
        reservoir = system.nodes[reservoir_id]

        flow_trace = turbine.trace(WaterPassedThrough)
        stored_trace = reservoir.trace(WaterStored)
        released_trace = reservoir.trace(WaterReleased)

        net_change = stored_trace - released_trace
        storage_trace = net_change.cumsum(initial=initial_storage)
        head_trace = storage_trace.map(volume_to_head_lifted)

        power_trace = flow_trace * head_trace * (9810 * efficiency / 1e9)
        return power_trace.sum()

    return Objective(name="hydropower", direction="maximize", evaluate=evaluate)


def flood_objective(node_id: str) -> Objective:
    """Create objective to minimize flood risk (spillage) at a passthrough node."""

    def evaluate(system: WaterSystem) -> float:
        return system.nodes[node_id].trace(WaterSpilled).sum()

    return Objective(name="city_flood", direction="minimize", evaluate=evaluate)


def deficit_objective(node_id: str, name: str) -> Objective:
    """Create objective to minimize water deficit at a demand node."""

    def evaluate(system: WaterSystem) -> float:
        return system.nodes[node_id].trace(DeficitRecorded, field="deficit").sum()

    return Objective(name=name, direction="minimize", evaluate=evaluate)

## System Construction

Build the water network with 9 nodes and 8 edges:

```
        [river]
           ↓
      [reservoir]
           ↓
       [turbine]  ← Hydropower generation
           ↓
        [city]    ← PassThrough with capacity limit (flood risk)
           ↓
       [splitter]
         ↙    ↘
[irrigation]  [thermal_plant]  ← Competing demands
      ↓              ↓
 [irr_sink]    [thermal_sink]
```

Key constraints:
- **city capacity = 41**: Flow exceeding this causes flooding
- **irrigation**: Consumptive agricultural demand (seasonal)
- **thermal_plant**: Non-consumptive cooling demand (passes water through)

In [ ]:
# Create strategy instances
slop_rule = SLOPRelease()
seasonal_ratio = SeasonalRatio()


@dataclass(frozen=True)
class ZeroEdgeLoss:
    """Edge loss rule with no losses."""

    def calculate(self, edge: Edge, flow: float, t: int, dt: float) -> dict[LossReason, float]:
        return {}


@dataclass(frozen=True)
class ZeroStorageLoss:
    """Storage loss rule with no losses."""

    def calculate(self, node: Storage, t: int, dt: float) -> dict[LossReason, float]:
        return {}


zero_edge_loss = ZeroEdgeLoss()
zero_storage_loss = ZeroStorageLoss()

# Build system
system = WaterSystem(dt=1.0)

# Add nodes
system.add_node(Source(id="river", inflow=TimeSeries(inflows)))
system.add_node(
    Storage(
        id="reservoir",
        capacity=150.0,
        dead_storage=10.0,
        initial_storage=75.0,
        release_rule=slop_rule,
        loss_rule=zero_storage_loss,
    )
)
system.add_node(PassThrough(id="turbine", capacity=60.0))
system.add_node(PassThrough(id="city", capacity=CITY_CAPACITY))  # Flood risk bottleneck
system.add_node(Splitter(id="splitter", split_rule=seasonal_ratio))
system.add_node(
    Demand(
        id="irrigation",
        requirement=TimeSeries(irrigation_demand),
        consumption_fraction=1.0,  # Fully consumptive
        efficiency=0.85,
    )
)
system.add_node(
    Demand(
        id="thermal_plant",
        requirement=TimeSeries(thermal_demand),
        consumption_fraction=0.0,  # Non-consumptive (cooling water passes through)
        efficiency=1.0,
    )
)
system.add_node(Sink(id="irrigation_sink"))
system.add_node(Sink(id="thermal_sink"))

# Add edges
edges = [
    ("e_river_res", "river", "reservoir", 200.0),
    ("e_res_turb", "reservoir", "turbine", 80.0),
    ("e_turb_city", "turbine", "city", 80.0),
    ("e_city_split", "city", "splitter", 80.0),
    ("e_split_irr", "splitter", "irrigation", 80.0),
    ("e_split_therm", "splitter", "thermal_plant", 80.0),
    ("e_irr_sink", "irrigation", "irrigation_sink", 80.0),
    ("e_therm_sink", "thermal_plant", "thermal_sink", 80.0),
]
for id_, src, tgt, cap in edges:
    system.add_edge(Edge(id=id_, source=src, target=tgt, capacity=cap, loss_rule=zero_edge_loss))

# Validate system structure
system.validate()
print("System validated successfully!")
print(f"Nodes: {len(system.nodes)}")
print(f"Edges: {len(system.edges)}")

In [ ]:
# Visualize input time series
fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

months = np.arange(TIMESTEPS)
years = months / 12

axes[0].plot(years, inflows, "b-", alpha=0.7)
axes[0].set_ylabel("Inflow (units)")
axes[0].set_title("River Inflow Time Series")
axes[0].grid(True, alpha=0.3)

axes[1].plot(years, irrigation_demand, "g-", alpha=0.7, label="Irrigation")
axes[1].plot(years, thermal_demand, "r-", alpha=0.7, label="Thermal")
axes[1].set_ylabel("Demand (units)")
axes[1].set_title("Demand Time Series")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

axes[2].plot(years, [sum(x) for x in zip(irrigation_demand, thermal_demand)], "purple", alpha=0.7)
axes[2].axhline(y=CITY_CAPACITY, color="orange", linestyle="--", label=f"City capacity ({CITY_CAPACITY})")
axes[2].set_ylabel("Total Demand (units)")
axes[2].set_xlabel("Time (years)")
axes[2].set_title("Total Downstream Demand vs City Capacity")
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Baseline Simulation

Run a single simulation with default parameters to establish baseline performance.

In [ ]:
# Run baseline simulation
system.simulate(timesteps=TIMESTEPS)

# Calculate baseline metrics
objectives = [
    hydropower_objective("reservoir", "turbine", initial_storage=75.0),
    flood_objective("city"),
    deficit_objective("irrigation", "irrigation_deficit"),
    deficit_objective("thermal_plant", "thermal_deficit"),
]

print("=== Baseline Performance ===")
for obj in objectives:
    score = obj.evaluate(system)
    print(f"{obj.name} ({obj.direction}): {score:.4f}")

# Show some trace statistics
reservoir = system.nodes["reservoir"]
turbine = system.nodes["turbine"]
city = system.nodes["city"]
irrigation = system.nodes["irrigation"]
thermal_plant = system.nodes["thermal_plant"]

print("\n=== System Statistics ===")
print(f"Turbine flow - Total: {turbine.trace(WaterPassedThrough).sum():.1f}")
print(f"City spillage (flood): {city.trace(WaterSpilled).sum():.1f}")
print(f"Irrigation deficit: {irrigation.trace(DeficitRecorded, field='deficit').sum():.1f}")
print(f"Thermal deficit: {thermal_plant.trace(DeficitRecorded, field='deficit').sum():.1f}")
print(f"Reservoir spillage: {reservoir.trace(WaterSpilled).sum():.1f}")

## Multi-Objective Optimization

Run NSGA-II to find Pareto-optimal solutions balancing four competing objectives:
- Maximize hydropower generation
- Minimize city flooding
- Minimize irrigation deficit
- Minimize thermal plant deficit

The optimizer will tune all 72 parameters (60 SLOP + 12 SeasonalRatio) to find trade-off solutions.

In [ ]:
# Reset system for optimization
system.reset()

# Run optimization
result = optimize(
    system=system,
    objectives=[
        hydropower_objective("reservoir", "turbine", initial_storage=75.0),
        flood_objective("city"),
        deficit_objective("irrigation", "irrigation_deficit"),
        deficit_objective("thermal_plant", "thermal_deficit"),
    ],
    timesteps=TIMESTEPS,
    pop_size=100,
    generations=50,
    seed=SEED,
    verbose=True,
)

print(f"\nOptimization complete!")
print(f"Found {len(result.solutions)} Pareto-optimal solutions")

In [ ]:
# Extract objective scores
hp = [s.scores["hydropower"] for s in result.solutions]
fl = [s.scores["city_flood"] for s in result.solutions]
ir = [s.scores["irrigation_deficit"] for s in result.solutions]
th = [s.scores["thermal_deficit"] for s in result.solutions]

# Create 2x2 pairwise Pareto front plots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Hydropower vs City Flood
sc1 = axes[0, 0].scatter(hp, fl, c=ir, cmap="viridis", s=30, alpha=0.7)
axes[0, 0].set_xlabel("Hydropower (GWh) - maximize →")
axes[0, 0].set_ylabel("← minimize - City Flood Risk")
axes[0, 0].set_title("Hydropower vs City Flood\n(color: irrigation deficit)")
plt.colorbar(sc1, ax=axes[0, 0], label="Irrigation Deficit")
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Irrigation Deficit vs Thermal Deficit
sc2 = axes[0, 1].scatter(ir, th, c=hp, cmap="plasma", s=30, alpha=0.7)
axes[0, 1].set_xlabel("← minimize - Irrigation Deficit")
axes[0, 1].set_ylabel("← minimize - Thermal Deficit")
axes[0, 1].set_title("Irrigation vs Thermal Deficit\n(color: hydropower)")
plt.colorbar(sc2, ax=axes[0, 1], label="Hydropower")
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Hydropower vs Irrigation Deficit
sc3 = axes[1, 0].scatter(hp, ir, c=th, cmap="coolwarm", s=30, alpha=0.7)
axes[1, 0].set_xlabel("Hydropower (GWh) - maximize →")
axes[1, 0].set_ylabel("← minimize - Irrigation Deficit")
axes[1, 0].set_title("Hydropower vs Irrigation Deficit\n(color: thermal deficit)")
plt.colorbar(sc3, ax=axes[1, 0], label="Thermal Deficit")
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: City Flood vs Total Deficit
total_deficit = [i + t for i, t in zip(ir, th)]
sc4 = axes[1, 1].scatter(fl, total_deficit, c=hp, cmap="plasma", s=30, alpha=0.7)
axes[1, 1].set_xlabel("← minimize - City Flood Risk")
axes[1, 1].set_ylabel("← minimize - Total Deficit (Irr + Thermal)")
axes[1, 1].set_title("City Flood vs Total Deficit\n(color: hydropower)")
plt.colorbar(sc4, ax=axes[1, 1], label="Hydropower")
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics
print("\n=== Pareto Front Statistics ===")
print(f"Hydropower: min={min(hp):.2f}, max={max(hp):.2f}, range={max(hp) - min(hp):.2f}")
print(f"City Flood: min={min(fl):.1f}, max={max(fl):.1f}, range={max(fl) - min(fl):.1f}")
print(f"Irrigation Deficit: min={min(ir):.1f}, max={max(ir):.1f}, range={max(ir) - min(ir):.1f}")
print(f"Thermal Deficit: min={min(th):.1f}, max={max(th):.1f}, range={max(th) - min(th):.1f}")

## Conclusion

This notebook demonstrated multi-objective optimization of a water system with:

1. **72 tunable parameters** across two time-varying strategies (SLOP release rule + seasonal split)
2. **Four competing objectives** with genuine trade-offs:
   - Hydropower vs City Flood (release rate trade-off)
   - Irrigation vs Thermal (water allocation trade-off)
3. **NSGA-II optimization** finding Pareto-optimal solutions

**Key trade-offs revealed:**
- Higher hydropower generation requires more water release, risking city flooding
- Irrigation and thermal demands compete for the same water after the city bottleneck
- Reducing one deficit increases the other - a zero-sum allocation problem

Decision makers can select solutions from the Pareto front based on their priorities:
- Prioritize food security → favor irrigation
- Prioritize energy production → favor hydropower and thermal
- Prioritize flood safety → accept lower hydropower output